In [1]:
import tensorflow as tf
import pickle
import numpy as np
import timeit

from load_cifar import *

In [2]:
tf.reset_default_graph()

<h1>Define Hyper-perparmeter</h1>

In [9]:
#learning rate
lr = 0.01
#number of traning steps
num_epochs =10

#number of batch_size
batch_size = 128

#network parameters
n_hidden_1 = 300
n_hidden_2 = 100
num_classes = 10

<h1>Define Placeholder</h1>

In [4]:
X = tf.placeholder(tf.float32, shape=[None, 32, 32, 3], name='X')
Y = tf.placeholder(tf.float32, shape=[None, num_classes], name='Y')
Y_cls = tf.argmax(Y, dimension=1)

Instructions for updating:
Use the `axis` argument instead


<h1>Define Neural Network Architecture</h1>

In [12]:
def fully_connected_net(x):
    droup_out_prob = 0.8
    flat = tf.contrib.layers.flatten(x)  

    full1 = tf.contrib.layers.fully_connected(inputs=flat, num_outputs=128, activation_fn=tf.nn.relu)
    full1 = tf.nn.dropout(full1, droup_out_prob)
    full1 = tf.layers.batch_normalization(full1)
    
    # 11
    full2 = tf.contrib.layers.fully_connected(inputs=full1, num_outputs=256, activation_fn=tf.nn.relu)
    full2 = tf.nn.dropout(full2, droup_out_prob)
    full2 = tf.layers.batch_normalization(full2)
    
    # 12
    full3 = tf.contrib.layers.fully_connected(inputs=full2, num_outputs=512, activation_fn=tf.nn.relu)
    full3 = tf.nn.dropout(full3, droup_out_prob)
    full3 = tf.layers.batch_normalization(full3)    
    
    # 13
    full4 = tf.contrib.layers.fully_connected(inputs=full3, num_outputs=1024, activation_fn=tf.nn.relu)
    full4 = tf.nn.dropout(full4, droup_out_prob)
    full4 = tf.layers.batch_normalization(full4)        
    
#     # 14
    out = tf.contrib.layers.fully_connected(inputs=full4, num_outputs=10, activation_fn=None)
    return out


<h1>Define cost andoptimization</h1>

In [13]:
#predicted labels
logits = fully_connected_net(X)
#define loss

# Loss and Optimizer
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))

train_optimizer = tf.train.AdamOptimizer(learning_rate=lr).minimize(loss)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32),name='accuracy')


<h1>Training and testing</h1>
<h2>1.Print out validation accuracy after each training poch</h2>
<h2>2.Print out training time you spend on each epoch</h2>
<h2>3.Print out testing accuracy in the end</h2>

In [15]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    #Initialize the variables
    sess.run(init)
    #fetch batch
    for i in range(num_epochs):
        for fearues_batch, labels_batch in load_preprocessed_training_batch(1,batch_size):
            #run optimization
            sess.run(train_optimizer, feed_dict={X:fearues_batch, Y:labels_batch})

        acc = sess.run(accuracy,feed_dict={X:fearues_batch, Y:labels_batch})
        print("step "+str(i)+", Accuracy= {:.3f}".format(acc))
    print("Training finished!") 
    test_features, test_labels = pickle.load(open('cifar-preprocessed/batch_test','rb'))
    print("Testing ACcuracy:", sess.run(accuracy, feed_dict={X:test_features, Y:test_labels}))


step 0, Accuracy= 0.180
step 1, Accuracy= 0.266
step 2, Accuracy= 0.250
step 3, Accuracy= 0.250
step 4, Accuracy= 0.312
step 5, Accuracy= 0.367
step 6, Accuracy= 0.367
step 7, Accuracy= 0.484
step 8, Accuracy= 0.414
step 9, Accuracy= 0.297
Training finished!
Testing ACcuracy: 0.1409
